In [15]:
'''
Caleb Haynes 6/8/2020 for David Smith & Johanna Jarcho
Converts a set of behavioral text files in the AFNI format (seperated txt files) into one 'BIDS' standard file per participant
'''
import pandas as pd
import glob
import csv
from numpy import loadtxt

In [16]:
#make a list of all subject level directories to parse over
sub_file_list = glob.glob('*/*')

In [17]:
'''
main function to call in loop, converts a text file to a pandas dataframe with the BIDS format we want with columns for onset, 
duration and trial_type. We will want to join the dataframes per participant in the loop so we can sort by onset and maintain 
correct durations and task_types. 
''' 
#txt path string, duration int, trial_string
def fp_to_pd(txt_path, duration, trial_type):
        #I use glob.glob in the loop to find the file. This creates a list object so we use first item in list
        #([0]) and use numpy's loadtext module to grab the data inside the txt file. 
        txt_path = loadtxt(txt_path[0], delimiter=" ")
        #create an initial dictionary to load into pandas with our first column
        txt_dict = {'onset': txt_path}
        #instantiate pandas df with our dict
        df = pd.DataFrame(data = txt_dict)
        #add column for duration values
        df['duration'] = duration
        #add column with the trial_type
        df['trial_type'] = trial_type
        return df

In [20]:
for i in sub_file_list:
    #itertate over each subject, make a path string for each type of file, here decision, positive feedback, and negative 
    #feedback. 
    f_social_dec = glob.glob(i + '/social/decision/decisionPhase_social_*')
    print(i)
    print(f_social_dec)
    f_social_fb_pos = glob.glob(i + '/social/feedback/pos*')
    f_social_fb_neg = glob.glob(i + '/social/feedback/neg*')
    
    f_doors_dec = glob.glob(i + '/doors/decision/decisionPhase_doors_*')
    f_doors_fb_pos = glob.glob(i + '/doors/feedback/pos*')
    f_doors_fb_neg = glob.glob(i + '/doors/feedback/neg*')
    
    #call function with each file path string, duration, and trial_type
    s_df_dec = fp_to_pd(f_social_dec, 3, 'decision')
    s_df_fb_pos = fp_to_pd(f_social_fb_pos, 1, 'win')
    s_df_fb_neg = fp_to_pd(f_social_fb_neg, 1, 'loss')
    
    d_df_dec = fp_to_pd(f_doors_dec, 3, 'decision')
    d_df_fb_pos = fp_to_pd(f_doors_fb_pos, 1, 'win')
    d_df_fb_neg = fp_to_pd(f_doors_fb_neg, 1, 'loss')
    
    #list of dataframes, merge together
    s_df_list = [s_df_dec,s_df_fb_pos,s_df_fb_neg]
    s_df_merge = pd.concat(s_df_list)
    
    d_df_list = [d_df_dec,d_df_fb_pos,d_df_fb_neg]
    d_df_merge = pd.concat(d_df_list)
    #sort rows by onset values and reset index
    df_soc = s_df_merge.sort_values(by=['onset']).reset_index()
    df_doors = d_df_merge.sort_values(by=['onset']).reset_index()
    
    soc_filename = f'sub-{i[-3:]}_face-A.tsv'
    door_filename = f'sub-{i[-3:]}_image-A.tsv'
    
    df_soc.to_csv(soc_filename, columns = ['onset', 'duration', 'trial_type'],index =False, sep ='\t')
    df_doors.to_csv(door_filename, columns = ['onset', 'duration', 'trial_type'],index =False, sep ='\t')

oa_onsets/p001
['oa_onsets/p001/social/decision/decisionPhase_social_p001.txt']
oa_onsets/p002
['oa_onsets/p002/social/decision/decisionPhase_social_p002.txt']
oa_onsets/p003
['oa_onsets/p003/social/decision/decisionPhase_social_p003.txt']
oa_onsets/p004
['oa_onsets/p004/social/decision/decisionPhase_social_p004.txt']
oa_onsets/p005
['oa_onsets/p005/social/decision/decisionPhase_social_p005.txt']
oa_onsets/p006
['oa_onsets/p006/social/decision/decisionPhase_social_p006.txt']
ra_onsets/s523
['ra_onsets/s523/social/decision/decisionPhase_social_s523.txt']
ra_onsets/s571
['ra_onsets/s571/social/decision/decisionPhase_social_s571.txt']
ra_onsets/s578
['ra_onsets/s578/social/decision/decisionPhase_social_s578.txt']
ra_onsets/s606
['ra_onsets/s606/social/decision/decisionPhase_social_s606.txt']
ra_onsets/s707
['ra_onsets/s707/social/decision/decisionPhase_social_s707.txt']
ra_onsets/s730
['ra_onsets/s730/social/decision/decisionPhase_social_s730.txt']


In [19]:
!ls


afni_to_BIDS.ipynb   sub-004_face-A.tsv   sub-571_image-A.tsv
oa_onsets	     sub-004_image-A.tsv  sub-578_face-A.tsv
ra_onsets	     sub-005_face-A.tsv   sub-578_image-A.tsv
sub-001_face-A.tsv   sub-005_image-A.tsv  sub-606_face-A.tsv
sub-001_image-A.tsv  sub-006_face-A.tsv   sub-606_image-A.tsv
sub-002_face-A.tsv   sub-006_image-A.tsv  sub-707_face-A.tsv
sub-002_image-A.tsv  sub-523_face-A.tsv   sub-707_image-A.tsv
sub-003_face-A.tsv   sub-523_image-A.tsv  sub-730_face-A.tsv
sub-003_image-A.tsv  sub-571_face-A.tsv   sub-730_image-A.tsv
